In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from collections import defaultdict

from pathlib import Path, PosixPath
from PIL import Image

import numpy as np
import cv2

from tqdm import tqdm

import matplotlib.pyplot as plt

from IPython.display import FileLink 

from typing import Optional

import shutil

In [3]:
VIDEO_FPS = 25
FRAME_SIZE = (1920, 1080)
UNDERSCORE_SYMBOL = '_'

In [4]:
def transform_bbox_coordinates(coordinates: list, height: int, width: int) -> Optional[tuple]:
    if coordinates:
        x_1 = int(width * coordinates[1] - width * coordinates[3] / 2)
        y_1 = int(height * coordinates[2] - height * coordinates[4] / 2)
        x_2 = int(width * coordinates[1] + width * coordinates[3] / 2)
        y_2 = int(height * coordinates[2] + height * coordinates[4] / 2)
        return x_1, y_1, x_2, y_2
    return None

In [5]:
def get_bounding_boxes(labels_file_path: Path, height: int, width: int) -> dict:
    res = dict()
    for fname in labels_file_path.iterdir():
        with open(fname, 'r') as f:
            data = list(map(float, f.readline().split()))
        res[fname.stem] = transform_bbox_coordinates(data, height, width)
    return res

In [6]:
def make_video_from_images(images_file_paths: list, 
                           file_name: str, 
                           bounding_boxes=dict(),
                           fps: int = VIDEO_FPS,
                           size: tuple = FRAME_SIZE):
    
    video_cod = cv2.VideoWriter_fourcc(*'XVID')
    writer = cv2.VideoWriter(f'/kaggle/working/{file_name}.avi',video_cod, VIDEO_FPS, FRAME_SIZE)
    
    for fname in tqdm(sorted(images_file_paths, key=lambda x: int(x.stem.split(UNDERSCORE_SYMBOL)[0])), 
                     total=len(images_file_paths), desc=f'{file_name}'):
        frame = cv2.imread(str(fname), cv2.IMREAD_COLOR)
        frame = cv2.resize(frame, FRAME_SIZE)
        try:
            (x_1, y_1, x_2, y_2) = bounding_boxes[fname.stem]
        except KeyError:
            pass
        else:
            cv2.rectangle(frame, (x_1, y_1), (x_2, y_2), (0, 0, 255), 3)
            
        writer.write(frame)
        
    writer.release()

In [7]:
train_dir_images = Path('/kaggle/input/table-tennis-ball-position-detection-dataset/openttgames/openttgames/images/train')
test_dir_images = Path('/kaggle/input/table-tennis-ball-position-detection-dataset/openttgames/openttgames/images/test')

train_dir_labels = Path('/kaggle/input/table-tennis-ball-position-detection-dataset/openttgames/openttgames/labels/train')
test_dir_labels = Path('/kaggle/input/table-tennis-ball-position-detection-dataset/openttgames/openttgames/labels/test')

# Amount of train and test images
train_images_count = len(list(train_dir_images.glob('*.png')))
test_images_count = len(list(test_dir_images.glob('*.png')))

# Amount of train and test labels
train_labels_count = len(list(train_dir_labels.glob('*.txt')))
test_labels_count = len(list(test_dir_labels.glob('*.txt')))

print(f'Amount of train images: {train_images_count}')
print(f'Amount of test images: {test_images_count}')
print(f'Overall: {train_images_count + test_images_count}')
print('---------------------')
print(f'Amount of train labels files: {train_labels_count}')
print(f'Amount of test labels files: {test_labels_count}')
print(f'Overall: {train_labels_count + test_labels_count}')

In [8]:
# Separation train and test image files by underscore symbol entering
train_images_with_underscore = list()
train_images_without_underscore = list()

test_images_with_underscore = list()
test_images_without_underscore = list()

for file_name in train_dir_images.iterdir():
    if UNDERSCORE_SYMBOL in file_name.stem:
        train_images_with_underscore.append(file_name)
    else:
        train_images_without_underscore.append(file_name)
        
for file_name in test_dir_images.iterdir():
    if UNDERSCORE_SYMBOL in file_name.stem:
        test_images_with_underscore.append(file_name)
    else:
        test_images_without_underscore.append(file_name)
        
print(f'Train images with underscore: {len(train_images_with_underscore)}')
print(f'Train images without underscore: {len(train_images_without_underscore)}')
print(f'Overall: {len(train_images_with_underscore) + len(train_images_without_underscore)}')
print('----------------------')
print(f'Test images with underscore: {len(test_images_with_underscore)}')
print(f'Test images without underscore: {len(test_images_without_underscore)}')
print(f'Oversll: {len(test_images_with_underscore) + len(test_images_without_underscore)}')

In [9]:
# Separation train and test image files by game number
train_games_dict = defaultdict(list)
test_games_dict = defaultdict(list)

for fname in train_images_with_underscore:
    game_number = fname.stem.split(UNDERSCORE_SYMBOL)[1]
    train_games_dict[game_number].append(fname)

train_games_dict['0'] = train_images_without_underscore

for fname in test_images_with_underscore:
    game_number = fname.stem.split(UNDERSCORE_SYMBOL)[1]
    test_games_dict[game_number].append(fname)
    
test_games_dict['0'] = test_images_without_underscore

print(f'Train game numbers: {list(train_games_dict.keys())}')
print(f'Test game numbers: {list(test_games_dict.keys())}')

In [16]:
train_bounding_boxes = get_bounding_boxes(train_dir_labels, FRAME_SIZE[1], FRAME_SIZE[0])
test_bounding_boxes = get_bounding_boxes(test_dir_labels, FRAME_SIZE[1], FRAME_SIZE[0])

In [42]:
for game, fnames in train_games_dict.items():
    make_video_from_images(fnames, f'train_game_{game}', train_bounding_boxes)
    
for game, fnames in test_games_dict.items():
    make_video_from_images(fnames, f'test_game_{game}', test_bounding_boxes)

In [53]:
from IPython.display import FileLink 
FileLink(r'test_game_3.avi')

# Check for maximum consecutive images

In [10]:
def get_max_consecutive_interval(paths: list) -> tuple:
    sorted_frames = sorted([int(x.stem.split(UNDERSCORE_SYMBOL)[0]) for x in paths])
    
    start_frame = sorted_frames[0]
    counter = 1
    total_counter = 1
    total_start_frame = 1
    
    for frame_number in sorted_frames[1:]:
        if frame_number == (start_frame + counter):
            counter += 1
            if counter > total_counter:
                total_counter = counter
                total_start_frame = start_frame
            continue
        start_frame = frame_number
        counter = 1
        
    return total_start_frame, total_counter

In [11]:
max_consecutive_intervals = dict()

for game, fnames in train_games_dict.items():
    max_consecutive_intervals[game] = get_max_consecutive_interval(fnames)
    
max_consecutive_intervals

In [22]:
# Does not consecutive frames 
max_interval_frames = [train_dir_images / f'{x}.png' for x in range(max_consecutive_intervals['0'][0],
                                                                    32)]

In [20]:
max_interval_labels = [train_dir_labels / f'{x}.txt' for x in range(max_consecutive_intervals['0'][0],
                                                                    32)]
max_interval_labels

In [24]:
for fname in max_interval_frames:
    shutil.copy(fname, '/kaggle/working/')

In [17]:
make_video_from_images(max_interval_frames, 'max_length_interval')